In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

from dataset import load_dataset
from models import ALEXNET, VGGNET, LENET

def show_result(images, labels, test_pred, i) :
    plt.imshow(images[i], interpolation='nearest')
    plt.axis('off')
    plt.show()

    target = np.argmax(labels[i])
    if target == 11:
        print("Target: More than 10 items")
    else :
        print("Target: " + str(target) + " items")

    pred = np.argmax(test_pred[i])
    if target == 11:
        print("Prediction: More than 10 items")
    else :
        print("Prediction: %d items by %.2f percent" % (pred, (test_pred[i][pred])*100))

def test(model, sess, saver, test_data, function, difficulty, batch_size):
    """
    Tester
    """
    batch_image, batch_image_index = test_data.get_batch_tensor(batch_size=batch_size)

    with tf.Session() as _sess:
        _sess.run(tf.local_variables_initializer())
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=_sess, coord=coord)

        images, indices = _sess.run([batch_image, batch_image_index])
        labels = test_data.get_labels_from_indices(indices, function, difficulty)
        test_metric, test_pred = model.eval_metric(images, labels)
        
        show_result(images, labels, test_pred, 0)
        show_result(images, labels, test_pred, 1)
        show_result(images, labels, test_pred, 2)
        show_result(images, labels, test_pred, 3)
        show_result(images, labels, test_pred, 4)
        
        coord.request_stop()
        coord.join(threads)

def run(model_name, function, difficulty, batch_size, learning_rate = 0.0025) :
    with tf.Session() as sess:
            # Define computation graph & Initialize
            print('Building network & initializing variables')
            if model_name == 'ALEXNET':
                model = ALEXNET(function, learning_rate, difficulty)
            elif model_name == 'VGGNET':
                model = VGGNET(function, learning_rate, difficulty)
            else:
                model = LENET(function, learning_rate, difficulty)

            model.init_sess(sess)
            saver = tf.train.Saver()

            # Process data
            print("Load dataset")
            dataset = load_dataset()
            test_data = dataset.test

            print('Loading best checkpointed model')
            saver.restore(sess, "../"+model.model_filename)
            test(model, sess, saver, test_data, function, difficulty, batch_size)

In [ ]:
run("VGGNET", "count", "moderate", 98)